# NEPOOL IDR Drop

Batches tasks for NEPOOL IDR Drop.

In [1]:
import os
basepath = '/home/jupyter-engiela/la-tools-test/IDR_Drop/'
os.chdir(basepath)
print('working in directory {}.'.format(basepath))

from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
import selenium.webdriver.support.ui as ui
from selenium.webdriver.common.keys import Keys
import selenium.webdriver as webdriver
from pandas.io.json import json_normalize
import datetime as dt
import pandas as pd
import numpy as np
import IDRdrop
import EPOwebscrape
#import emailscrape
import json
import math
import time

working in directory /home/jupyter-engiela/la-tools-test/IDR_Drop/.


## Convert .json to .xlsx

In [2]:
with open(filename, 'r') as f:
    bodies = json.load(f)

    bodies = json.loads(bodies)

good = EPOwebscrape.bodies_json(bodies)

print('converted .json to dataframe')

converted .json to dataframe


In [4]:
outfile = filename.split('.')[0]
outfile = '.'.join([outfile, 'xlsx'])
good.to_excel(outfile, index = False)
print('wrote {} to {}'.format(outfile, basepath))

wrote email_bodies_09_30_2019.xlsx to /home/jupyter-engiela/la-tools-test/IDR_Drop/


## Read Excel file

In [2]:
good = pd.read_excel('email_bodies_10_11_2019__20191013210226.xlsx')
good

accts                date  \
0  ['51420574014_465861007', '51452223035_0408120... 2019-10-11 11:36:04   
1                          ['54202385001_052931000'] 2019-10-11 11:34:00   
2  ['51420574014_465861007', '51452223035_0408120... 2019-10-11 10:34:43   
3  ['54977381078_126733008', '54480402023_5265410... 2019-10-10 15:55:06   
4                          ['51436492052_684002003'] 2019-10-10 15:54:30   
5  ['51549893006_482562007', '51874553092_8600420... 2019-10-10 09:43:48   
6                          ['51482323003_190712004'] 2019-10-09 13:12:03   

                              name     pw          user   util  
0  Hartford Healthcare Corporation  78cck   harth-engie    CLP  
1       Protective Armored Systems  m372b    usar-engie  WMECO  
2  Hartford Healthcare Corporation  78cck   harth-engie    CLP  
3    Berkshire Medical Center Inc.  d823k  berkmc-engie  WMECO  
4                 Teds Food Center  tc836    teds-engie    CLP  
5          PM Engineered Solutions  p899g     pme-engie    CLP  
6                   Bimba Bakeries  9567c    bbak-engie    CLP

## Filter downloaded files from EPO portal

In [3]:
results = []
beg = time.time()

for row in range(0, len(good.accts)):
    try:
        EPOwebscrape.idr_download(row, good)
        results.append(True)
        print('done with {} of {}.'.format(row+1, len(good.accts)))
        print('---------------------------')
    except:
        print('error, accts {}.'.format(good.accts[row]))
        results.append(False)
        print('done with {} of {}.'.format(row+1, len(good.accts)))
        print('---------------------------')

end = time.time()
elapse = round(end - beg, 2)
print('downloaded {} files in {} seconds.'.format(len(good.accts), elapse))
good['success'] = results

user:  harth-engie
password:  78cck
logging on...
set to last  400  days.
logging on...
looking for accts ['51420574014_465861007', '51452223035_040812006', '51472893064_344143007', '51492834064_460192005', '51505275024_323761006', '51511694010_317103008', '51546023086_133612007', '51601665094_750042003', '51640724076_352382007', '51733934061_347092002', '51801694019_298103007', '51970204004_216762005', '51997893060_648243008'].
found items in portal.
trying search & download...


/home/jupyter-engiela/la-tools-test/IDR_Drop/EPOwebscrape.py:196: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 196 of the file /home/jupyter-engiela/la-tools-test/IDR_Drop/EPOwebscrape.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(browser.page_source)


disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['423281', '423229', '423271', '423284'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['423232', '420714', '420677', '423218'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['187763', '423281', '423265', '423259'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['423328', '423271', '423284', '423275'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['420714', '420721', '423211', '423268'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['423278', '423281', '423229', '423262'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['423284', '423224', '423232', '423275'].
disabling demand...


exported ['423229', '423262', '423328', '423271'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['423284', '423224', '423232', '423275'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['420714', '420718', '420721', '420677'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['423221', '423211', '423218', '423325'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['187763', '423235', '423268', '423278'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['423281', '423265', '423259', '423229'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['423262', '423328', '423271', '423284'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['423224', 

downloaded EPO data file.
exported ['423218', '423235', '423268', '423278'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['423281', '423265', '423259', '423229'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['423262', '423328', '423271', '423284'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['423224', '423232', '423275', '420714'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['420718', '420721', '420677', '423221'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['423211', '423218', '423325', '187763'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['423235', '423268', '423328', '423271'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data 

...

exported ['423259', '423328', '423271', '423284'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['423275', '420714', '420721', '423211'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['423268', '423278', '423281', '423265'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['423229', '423262', '423271', '423275'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['423221', '423325', '187763', '423235'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['423281', '423265', '423259', '423328'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['423271', '423284', '423275', '420714'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['420721', 

downloaded EPO data file.
exported ['423265', '423259', '423229', '423262'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['423328', '423271', '423284', '423224'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['423232', '423275', '420714', '420718'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['420721', '420677', '423221', '423211'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['423218', '423325', '187763', '423235'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['423268', '423328', '423271', '423224'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['423232', '420718', '420721', '420677'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data 

disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['423235', '423268', '423281', '423265'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['423259', '423328', '423271', '423284'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['423275', '420714', '420721', '423211'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['423268', '423265', '423259', '423262'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['423284', '423224', '423232', '423275'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['420714', '420718', '420721', '423221'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['423325', '187763', '423235', '423265'].
disabling demand...


exported ['423224', '423232', '423275', '420718'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['420677', '423221', '423211', '423218'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['423325', '187763', '423268', '423281'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['423229', '423271', '423284', '423232'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['420714', '420677', '423218', '187763'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['423281', '423265', '423259', '423328'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['423271', '423284', '423275', '420714'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['420721', 

downloaded EPO data file.
exported ['423259', '423229', '423262', '423224'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['423232', '420718', '420677', '423221'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['423218', '423235', '423268', '423278'].
disabling demand...
selecting hourly interval...
submitting...
downloaded EPO data file.
exported ['423281', '423265', '423259', '423229'].
error, accts ['51420574014_465861007', '51452223035_040812006', '51472893064_344143007', '51492834064_460192005', '51505275024_323761006', '51511694010_317103008', '51546023086_133612007', '51601665094_750042003', '51640724076_352382007', '51733934061_347092002', '51801694019_298103007', '51970204004_216762005', '51997893060_648243008'].
done with 1 of 7.
---------------------------
error, accts ['54202385001_052931000'].
done with 2 of 7.
---------------------------
user:  harth-engie
password:  7

In [ ]:
print('success with {} of {} accounts.'.format(np.sum(good.success), len(good.success)))
good.to_csv('webscrape_bodies_093019.csv')
good

## Show downloaded files from EPO portal

Here *filepath* is a directory containing downloaded EPO files. Code will print 20 most recent files.

In [6]:
#filepath = os.getcwd()
readpath = os.path.join(basepath, 'Downloads')

myfiles = IDRdrop.show_dir(readpath, 22)
myfiles = myfiles.sort_values(by = 'time')
myfiles.reset_index(drop = True, inplace = True)

files found in dir:  /home/jupyter-engiela/la-tools-test/IDR_Drop/Downloads


In [7]:
master = pd.concat([good, myfiles.files], axis = 1)
master.head()

,accts,date,name,pw,user,util,success,files
0,"['51420574014_465861007', '51452223035_0408120...",2019-10-11 11:36:04,Hartford Healthcare Corporation,78cck,harth-engie,CLP,False,eversource_8e9b3a3c-07d2-4684-86b0-09b14e1021c...
1,['54202385001_052931000'],2019-10-11 11:34:00,Protective Armored Systems,m372b,usar-engie,WMECO,False,eversource_8e9b3a3c-07d2-4684-86b0-09b14e1021c...
2,"['51420574014_465861007', '51452223035_0408120...",2019-10-11 10:34:43,Hartford Healthcare Corporation,78cck,harth-engie,CLP,False,eversource_8e9b3a3c-07d2-4684-86b0-09b14e1021c...
3,"['54977381078_126733008', '54480402023_5265410...",2019-10-10 15:55:06,Berkshire Medical Center Inc.,d823k,berkmc-engie,WMECO,False,eversource_8e9b3a3c-07d2-4684-86b0-09b14e1021c...
4,['51436492052_684002003'],2019-10-10 15:54:30,Teds Food Center,tc836,teds-engie,CLP,False,eversource_8e9b3a3c-07d2-4684-86b0-09b14e1021c...


Choose files to split into Raw IDR files.

In [8]:
index = [0]

splitfiles = list(master.files)
print('files to split: ')
splitfiles

files to split: 


['eversource_8e9b3a3c-07d2-4684-86b0-09b14e1021cc_b0e2f8bd_hourlycsv.csv',
 'eversource_8e9b3a3c-07d2-4684-86b0-09b14e1021cc_0cf37acd_hourlycsv.csv',
 'eversource_8e9b3a3c-07d2-4684-86b0-09b14e1021cc_11782f0b_hourlycsv.csv',
 'eversource_8e9b3a3c-07d2-4684-86b0-09b14e1021cc_2bd4e4e9_hourlycsv.csv',
 'eversource_8e9b3a3c-07d2-4684-86b0-09b14e1021cc_d72eed99_hourlycsv.csv',
 'eversource_8e9b3a3c-07d2-4684-86b0-09b14e1021cc_05ac8bca_hourlycsv.csv',
 'eversource_8e9b3a3c-07d2-4684-86b0-09b14e1021cc_897f97e0_hourlycsv.csv',
 'eversource_8e9b3a3c-07d2-4684-86b0-09b14e1021cc_15b63bab_hourlycsv.csv',
 'eversource_8e9b3a3c-07d2-4684-86b0-09b14e1021cc_4376e066_hourlycsv.csv',
 'eversource_8e9b3a3c-07d2-4684-86b0-09b14e1021cc_487a9b4a_hourlycsv.csv',
 'eversource_8e9b3a3c-07d2-4684-86b0-09b14e1021cc_15a8c980_hourlycsv.csv',
 'eversource_8e9b3a3c-07d2-4684-86b0-09b14e1021cc_b75d0c15_hourlycsv.csv',
 'eversource_8e9b3a3c-07d2-4684-86b0-09b14e1021cc_6e45c8fc_hourlycsv.csv',
 'eversource_8e9b3a3c-07d

## Batch process downloaded EPO files into Raw IDRs

In [11]:
readpath = os.path.join(basepath, 'Downloads')
writepath = os.path.join(basepath, 'Raw_IDR')
error_log = []

for i, file in enumerate(master.files):
    IDRdrop.raw_split(file, readpath, writepath)
    print('success, file: ', file)

TypeError: raw_split() missing 2 required positional arguments: 'utility' and 'fname'

In [ ]:
IDRdrop.raw_split(filedf, readpath, writepath, utility, os.path.join(readpath, fname))
print('success, file: ', file)

## Show Raw IDR files based on utility

Here *rawpath* is directory containing Raw IDRs - 25 most recent will be shown.

In [37]:
rawfiles = IDRdrop.show_dir(writepath, 40)
print(rawfiles)

files found in dir:  /home/jupyter-engiela/la-tools-test/IDR_Drop/Raw_IDR
                           files                time
0    MECO_3105626009_IDR_RAW.csv 2019-09-30 18:17:42
1    MECO_2744086022_IDR_RAW.csv 2019-09-30 18:17:42
2    MECO_5136114027_IDR_RAW.csv 2019-09-30 18:17:42
3    MECO_6318393016_IDR_RAW.csv 2019-09-30 18:17:42
4    MECO_5261705038_IDR_RAW.csv 2019-09-30 18:17:42
5    MECO_1343548015_IDR_RAW.csv 2019-09-30 18:17:42
6    MECO_2785791015_IDR_RAW.csv 2019-09-30 18:17:42
7    MECO_5188891013_IDR_RAW.csv 2019-09-30 18:17:42
8    MECO_1278508009_IDR_RAW.csv 2019-09-30 18:17:42
9    MECO_2835979089_IDR_RAW.csv 2019-09-30 18:17:42
10   MECO_7605460032_IDR_RAW.csv 2019-09-30 18:17:42
11   MECO_2542918027_IDR_RAW.csv 2019-09-30 18:17:42
12   MECO_3920288001_IDR_RAW.csv 2019-09-30 18:17:42
13   MECO_4081460053_IDR_RAW.csv 2019-09-30 18:17:42
14   MECO_7659209012_IDR_RAW.csv 2019-09-30 18:17:42
15   MECO_5190429018_IDR_RAW.csv 2019-09-30 18:17:42
16   MECO_0163357015_IDR_

## Transform vertical data to horizontal

In [39]:
index = [36]

processfiles = list(rawfiles.files[:37])
print('files to processed: ')
print(processfiles)

files to processed: 
['MECO_3105626009_IDR_RAW.csv', 'MECO_2744086022_IDR_RAW.csv', 'MECO_5136114027_IDR_RAW.csv', 'MECO_6318393016_IDR_RAW.csv', 'MECO_5261705038_IDR_RAW.csv', 'MECO_1343548015_IDR_RAW.csv', 'MECO_2785791015_IDR_RAW.csv', 'MECO_5188891013_IDR_RAW.csv', 'MECO_1278508009_IDR_RAW.csv', 'MECO_2835979089_IDR_RAW.csv', 'MECO_7605460032_IDR_RAW.csv', 'MECO_2542918027_IDR_RAW.csv', 'MECO_3920288001_IDR_RAW.csv', 'MECO_4081460053_IDR_RAW.csv', 'MECO_7659209012_IDR_RAW.csv', 'MECO_5190429018_IDR_RAW.csv', 'MECO_0163357015_IDR_RAW.csv', 'MECO_2810847013_IDR_RAW.csv', 'MECO_4027682019_IDR_RAW.csv', 'MECO_0353026025_IDR_RAW.csv', 'BECO_28276550010_IDR_RAW.csv', 'CLP_633303003_IDR_RAW.csv', 'CLP_23572076_IDR_RAW.csv', 'CLP_26016421054_IDR_RAW.csv', 'BECO_26569691004_IDR_RAW.csv', 'CLP_979927068_IDR_RAW.csv', 'BECO_28276470029_IDR_RAW.csv', 'BECO_28731780012_IDR_RAW.csv', 'BECO_16320950013_IDR_RAW.csv', 'CLP_30034800018_IDR_RAW.csv', 'CLP_688832070_IDR_RAW.csv', 'BECO_15956100034_IDR

,files,time
0,CLP_51482323003_190712004_IDR.csv,2019-10-14 23:54:52
1,CLP_51874553092_860042003_IDR_3.csv,2019-10-14 23:54:52
2,WMECO_54202385001_052931000_IDR.csv,2019-10-14 23:54:52
3,CLP_51970204004_216762005_IDR.csv,2019-10-14 23:54:52
4,CLP_51801694019_298103007_IDR.csv,2019-10-14 23:54:52
5,CLP_51733934061_347092002_IDR.csv,2019-10-14 23:54:52
6,CLP_51640724076_352382007_IDR_3.csv,2019-10-14 23:54:52
7,CLP_51640724076_352382007_IDR.csv,2019-10-14 23:54:52
8,CLP_51874553092_860042003_IDR.csv,2019-10-14 23:54:52
9,WMECO_54010781037_952531008_IDR.csv,2019-10-14 23:54:52


## Batch filter Raw IDR into IDR files to be dropped

In [40]:
readpath = os.path.join(basepath, 'Raw_IDR')
writepath = os.path.join(basepath, 'IDR')
error_log = []

for dropfile in processfiles:
    try:
        IDRdrop.data_drop(dropfile, readpath, writepath)
        print('success, file: ', dropfile)
    
    except:
        error_log.append(dropfile)
        print("error, file: ", dropfile)

success, file:  MECO_3105626009_IDR_RAW.csv
success, file:  MECO_2744086022_IDR_RAW.csv
success, file:  MECO_5136114027_IDR_RAW.csv
success, file:  MECO_6318393016_IDR_RAW.csv
success, file:  MECO_5261705038_IDR_RAW.csv
success, file:  MECO_1343548015_IDR_RAW.csv
success, file:  MECO_2785791015_IDR_RAW.csv
success, file:  MECO_5188891013_IDR_RAW.csv
success, file:  MECO_1278508009_IDR_RAW.csv
success, file:  MECO_2835979089_IDR_RAW.csv
success, file:  MECO_7605460032_IDR_RAW.csv
success, file:  MECO_2542918027_IDR_RAW.csv
success, file:  MECO_3920288001_IDR_RAW.csv
success, file:  MECO_4081460053_IDR_RAW.csv
success, file:  MECO_7659209012_IDR_RAW.csv
success, file:  MECO_5190429018_IDR_RAW.csv
success, file:  MECO_0163357015_IDR_RAW.csv
success, file:  MECO_2810847013_IDR_RAW.csv
success, file:  MECO_4027682019_IDR_RAW.csv
success, file:  MECO_0353026025_IDR_RAW.csv
success, file:  BECO_28276550010_IDR_RAW.csv
spot check output file at date: 2/27/2019.
success, file:  CLP_633303003_ID

In [41]:
idr = IDRdrop.show_dir(writepath, 15)
idr

[]

## Turn Horizontal to Vertical

In [44]:
idr = IDRdrop.show_dir(writepath, 37)

files found in dir:  /home/jupyter-engiela/la-tools-test/IDR_Drop/IDR


In [46]:
vert_files = list(idr.files[:37])
print('working with: ')
print(vert_files)

working with: 
['BECO_757226014_IDR_3.csv', 'BECO_757226014_IDR.csv', 'CLP_688832070_IDR_3.csv', 'BECO_15956100034_IDR.csv', 'BECO_28276550010_IDR.csv', 'CLP_979927068_IDR_3.csv', 'BECO_28072830012_IDR.csv', 'BECO_28276470029_IDR.csv', 'MECO_4027682019_IDR.csv', 'MECO_0163357015_IDR.csv', 'CLP_633303003_IDR.csv', 'CLP_998703078_IDR_3.csv', 'CLP_26016421054_IDR.csv', 'BECO_26569691004_IDR.csv', 'CLP_684104045_IDR.csv', 'BECO_16320950013_IDR.csv', 'CLP_534703046_IDR.csv', 'CLP_30034800018_IDR.csv', 'CLP_979927068_IDR.csv', 'MECO_0353026025_IDR.csv', 'BECO_28731780012_IDR.csv', 'CLP_633303003_IDR_3.csv', 'CLP_23572076_IDR.csv', 'CLP_998703078_IDR.csv', 'MECO_2810847013_IDR.csv', 'CLP_688832070_IDR.csv', 'MECO_1278508009_IDR.csv', 'MECO_2785791015_IDR.csv', 'MECO_5188891013_IDR.csv', 'MECO_2835979089_IDR.csv', 'MECO_7659209012_IDR.csv', 'MECO_5261705038_IDR.csv', 'MECO_5190429018_IDR.csv', 'MECO_6318393016_IDR.csv', 'MECO_2542918027_IDR.csv', 'MECO_1343548015_IDR.csv', 'MECO_3105626009_IDR

In [47]:
for v in vert_files:
    IDRdrop.hor_to_vert(v)

transformed and wrote BECO_757226014_IDR_3.csv to BECO_757226014_IDR_3_vert.csv.
transformed and wrote BECO_757226014_IDR.csv to BECO_757226014_IDR_vert.csv.
transformed and wrote CLP_688832070_IDR_3.csv to CLP_688832070_IDR_3_vert.csv.
transformed and wrote BECO_15956100034_IDR.csv to BECO_15956100034_IDR_vert.csv.
transformed and wrote BECO_28276550010_IDR.csv to BECO_28276550010_IDR_vert.csv.
transformed and wrote CLP_979927068_IDR_3.csv to CLP_979927068_IDR_3_vert.csv.
transformed and wrote BECO_28072830012_IDR.csv to BECO_28072830012_IDR_vert.csv.
transformed and wrote BECO_28276470029_IDR.csv to BECO_28276470029_IDR_vert.csv.
transformed and wrote MECO_4027682019_IDR.csv to MECO_4027682019_IDR_vert.csv.
transformed and wrote MECO_0163357015_IDR.csv to MECO_0163357015_IDR_vert.csv.
transformed and wrote CLP_633303003_IDR.csv to CLP_633303003_IDR_vert.csv.
transformed and wrote CLP_998703078_IDR_3.csv to CLP_998703078_IDR_3_vert.csv.
transformed and wrote CLP_26016421054_IDR.csv to

In [9]:
for file in idr.files:
    IDRdrop.hor_to_vert(file)

transformed and wrote CLP_51482323003_190712004_IDR.csv to CLP_51482323003_190712004_IDR_vert.csv.
transformed and wrote CLP_51874553092_860042003_IDR_3.csv to CLP_51874553092_860042003_IDR_3_vert.csv.
transformed and wrote WMECO_54202385001_052931000_IDR.csv to WMECO_54202385001_052931000_IDR_vert.csv.
transformed and wrote CLP_51970204004_216762005_IDR.csv to CLP_51970204004_216762005_IDR_vert.csv.
transformed and wrote CLP_51801694019_298103007_IDR.csv to CLP_51801694019_298103007_IDR_vert.csv.
transformed and wrote CLP_51733934061_347092002_IDR.csv to CLP_51733934061_347092002_IDR_vert.csv.
transformed and wrote CLP_51640724076_352382007_IDR_3.csv to CLP_51640724076_352382007_IDR_3_vert.csv.
transformed and wrote CLP_51640724076_352382007_IDR.csv to CLP_51640724076_352382007_IDR_vert.csv.
transformed and wrote CLP_51874553092_860042003_IDR.csv to CLP_51874553092_860042003_IDR_vert.csv.
transformed and wrote WMECO_54010781037_952531008_IDR.csv to WMECO_54010781037_952531008_IDR_vert